In [11]:
import os
import jax

from transformers import AutoTokenizer, CLIPProcessor
from configuration_hybrid_clip import HybridCLIPConfig
from modeling_hybrid_clip import FlaxHybridCLIP
from PIL import Image

import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.transforms.functional import InterpolationMode
from torchvision.transforms import Resize, Normalize, ConvertImageDtype, ToTensor
import numpy as np
from run_hybrid_clip import Transform

In [12]:
model = FlaxHybridCLIP.from_pretrained("flax-community/medclip-roco")

In [13]:
vision_model_name = "openai/clip-vit-base-patch32"
img_dir = "/Users/kaumad/Documents/coding/hf-flax/demo/medclip-roco/images"

processor = CLIPProcessor.from_pretrained(vision_model_name)

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [14]:
import pandas as pd

img_list = os.listdir(img_dir)
embeddings = []

for idx, img_path in enumerate(img_list):
    if idx % 50 == 0:
        print(f"{idx} images processed")
    img = Image.open(os.path.join(img_dir, img_path)).convert('RGB')
    inputs = processor(images=img, return_tensors="jax", padding=True)
    inputs['pixel_values'] = inputs['pixel_values'].transpose(0, 2, 3, 1)
    img_vec = model.get_image_features(**inputs)
    img_vec = np.array(img_vec).reshape(-1).tolist()
    embeddings.append(img_vec)
    # fvec.write(f"{img_path}\t{img_vec_s}\n")

0 images processed
50 images processed
100 images processed
150 images processed
200 images processed
250 images processed
300 images processed
350 images processed
400 images processed
450 images processed
500 images processed
550 images processed
600 images processed
650 images processed
700 images processed
750 images processed
800 images processed
850 images processed
900 images processed
950 images processed
1000 images processed
1050 images processed
1100 images processed
1150 images processed
1200 images processed
1250 images processed
1300 images processed
1350 images processed
1400 images processed
1450 images processed
1500 images processed
1550 images processed
1600 images processed
1650 images processed
1700 images processed
1750 images processed
1800 images processed
1850 images processed
1900 images processed
1950 images processed
2000 images processed
2050 images processed
2100 images processed
2150 images processed
2200 images processed
2250 images processed
2300 images

OSError: image file is truncated (8 bytes not processed)

In [15]:
np.array(embeddings).shape

(3385, 512)

In [11]:
query = 'carcinoma'
inputs = processor(text=[query], images=None, return_tensors="jax", padding=True)

query_vec = model.get_text_features(**inputs)
query_vec = np.asarray(query_vec)

In [30]:
query_vec = query_vec / np.linalg.norm(query_vec, axis=-1, keepdims=True)
embeddings = embeddings / np.linalg.norm(embeddings, axis=-1, keepdims=True)
products = np.sum(np.multiply(query_vec, embeddings), axis=1)
products.shape

(200,)

In [16]:
embeddings = np.array(embeddings)
embeddings = embeddings / np.linalg.norm(embeddings, axis=-1, keepdims=True)
embeddings = [np.asarray(embed) for embed in embeddings.tolist()]

In [19]:
len(embeddings)

3385

In [20]:
embeddings_df = pd.DataFrame({'files': img_list[:3385], 'image_embedding': embeddings})
pd.to_pickle(embeddings_df, "../../data/image_embeddings_large.pkl")

In [45]:
embeddings_df = pd.read_pickle("../../data/image_embeddings.pkl")

In [56]:
image_embeds = np.asarray(embeddings_df['image_embedding'])
image_list = np.asarray(embeddings_df['files'].tolist())

In [61]:
np.stack(embeddings_df['image_embedding']

(200, 512)

In [58]:
image_list[products.argsort()[-5:]]

array(['PMC2693557_jkms-22-159-g001.jpg',
       'PMC2553898_AU2008-918050.003.jpg',
       'PMC2684097_1749-7922-4-18-2.jpg', 'PMC2697535_tcrm-5-301f1.jpg',
       'PMC2438313_1757-1626-1-10-1.jpg'], dtype='<U40')